In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
os.chdir(os.path.join(os.path.dirname(os.getcwd()), 'data', 'in'))

In [3]:
clientes_df = pd.read_csv('train_clientes_sample.csv')
requerimientos_df = pd.read_csv('train_requerimientos_sample.csv')

In [4]:
def generar_variables_ingenieria(clientes_df):
    # SDO_ACTIVO
    clientes_df["VAR_SDO_ACTIVO_6M"] = clientes_df["SDO_ACTIVO_MENOS5"] - clientes_df["SDO_ACTIVO_MENOS0"]
    clientes_df["PROM_SDO_ACTIVO_0M_2M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(3)]].mean(axis=1)
    clientes_df["PROM_SDO_ACTIVO_3M_5M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(3, 6)]].mean(axis=1)
    clientes_df["VAR_SDO_ACTIVO_3M"] = clientes_df["PROM_SDO_ACTIVO_3M_5M"] - clientes_df["PROM_SDO_ACTIVO_0M_2M"]
    clientes_df["PROM_SDO_ACTIVO_6M"] = clientes_df[[f"SDO_ACTIVO_MENOS{i}" for i in range(6)]].mean(axis=1)

    # FLG_SEGURO
    clientes_df["MESES_CON_SEGURO"] = clientes_df[[f"FLG_SEGURO_MENOS{i}" for i in range(6)]].sum(axis=1)

    # CANALES
    for canal in [1, 2, 3]:
        base = f"NRO_ACCES_CANAL{canal}_MENOS"
        clientes_df[f"VAR_NRO_ACCES_CANAL{canal}_6M"] = clientes_df[f"{base}5"] - clientes_df[f"{base}0"]
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_6M"] = clientes_df[[f"{base}{i}" for i in range(6)]].mean(axis=1)
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_0M_2M"] = clientes_df[[f"{base}{i}" for i in range(3)]].mean(axis=1)
        clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_3M_5M"] = clientes_df[[f"{base}{i}" for i in range(3, 6)]].mean(axis=1)
        clientes_df[f"VAR_NRO_ACCES_CANAL{canal}_3M"] = (clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_3M_5M"] - clientes_df[f"PROM_NRO_ACCES_CANAL{canal}_0M_2M"])

    # ENTIDADES FINANCIERAS
    clientes_df["PROM_NRO_ENTID_SSFF_6M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(6)]].mean(axis=1)
    clientes_df["VAR_NRO_ENTID_SSFF_6M"] = clientes_df["NRO_ENTID_SSFF_MENOS5"] - clientes_df["NRO_ENTID_SSFF_MENOS0"]
    clientes_df["PROM_NRO_ENTID_SSFF_0M_2M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(3)]].mean(axis=1)
    clientes_df["PROM_NRO_ENTID_SSFF_3M_5M"] = clientes_df[[f"NRO_ENTID_SSFF_MENOS{i}" for i in range(3, 6)]].mean(axis=1)
    clientes_df["VAR_NRO_ENTID_SSFF_3M"] = (clientes_df["PROM_NRO_ENTID_SSFF_3M_5M"] - clientes_df["PROM_NRO_ENTID_SSFF_0M_2M"])

    # Saldo en otras entidades
    clientes_df["MESES_CON_SALDO"] = clientes_df[[f"FLG_SDO_OTSSFF_MENOS{i}" for i in range(6)]].sum(axis=1)

    return clientes_df

In [5]:
def imputacion_variables(clientes_df,requerimientos_df):
    moda = clientes_df['RANG_INGRESO'].mode()[0]
    clientes_df['RANG_INGRESO'].fillna(moda, inplace=True)

    moda = clientes_df['FLAG_LIMA_PROVINCIA'].mode()[0]
    clientes_df['FLAG_LIMA_PROVINCIA'].fillna(moda, inplace=True)

    clientes_df['EDAD'].fillna(clientes_df['EDAD'].median(), inplace=True)
    clientes_df['ANTIGUEDAD'].fillna(clientes_df['ANTIGUEDAD'].median(), inplace=True)

    moda = requerimientos_df['DICTAMEN'].mode()[0]
    requerimientos_df['DICTAMEN'].fillna(moda, inplace=True)

    return clientes_df, requerimientos_df

In [6]:
def encoder_categoricos(clientes_df):
    clientes_df['RANG_SDO_PASIVO_MENOS0'] = clientes_df['RANG_SDO_PASIVO_MENOS0'].replace('Cero', 'Rango_SDO_00')
    clientes_df['FLAG_LIMA_PROVINCIA'] = clientes_df['FLAG_LIMA_PROVINCIA'].map({'Lima': 1, 'Provincia': 0})
    cat_cols = clientes_df.select_dtypes(include=['object', 'category']).columns
    encoders_clientes = {} 

    for col in cat_cols:
        le = LabelEncoder()
        clientes_df[col] = le.fit_transform(clientes_df[col])
        encoders_clientes[col] = le

    return clientes_df, encoders_clientes

In [7]:
def construir_variables_requerimientos(df_reqs, id_col='ID_CORRELATIVO'):
    
    total_reqs = df_reqs.groupby(id_col).size().rename('total_requerimientos')
    if not isinstance(total_reqs, pd.DataFrame):
        total_reqs = total_reqs.to_frame()

    n_tipo_req = df_reqs.groupby(id_col)['TIPO_REQUERIMIENTO2'].nunique().rename('nro_tipos_requerimiento').to_frame()
    n_dictamen = df_reqs.groupby(id_col)['DICTAMEN'].nunique().rename('nro_dictamenes').to_frame()
    n_producto = df_reqs.groupby(id_col)['PRODUCTO_SERVICIO_2'].nunique().rename('nro_productos_servicios').to_frame()
    n_submotivo = df_reqs.groupby(id_col)['SUBMOTIVO_2'].nunique().rename('nro_submotivos').to_frame()

    tipo_ohe = pd.get_dummies(df_reqs['TIPO_REQUERIMIENTO2'], prefix='tipo')
    tipo_ohe[id_col] = df_reqs[id_col]
    tipo_ohe = tipo_ohe.groupby(id_col).sum()

    dictamen_ohe = pd.get_dummies(df_reqs['DICTAMEN'], prefix='dictamen')
    dictamen_ohe[id_col] = df_reqs[id_col]
    dictamen_ohe = dictamen_ohe.groupby(id_col).sum()

    df_agregado = pd.concat([total_reqs, n_tipo_req, n_dictamen, n_producto, n_submotivo, tipo_ohe, dictamen_ohe],axis=1)

    return df_agregado


In [8]:
def estandarizacion(df_final):
    no_escalar = ['ID_CORRELATIVO', 'CODMES', 'ATTRITION']

    columnas_a_escalar = df_final.columns.difference(no_escalar)

    df_predictoras = df_final[columnas_a_escalar]

    scaler = StandardScaler()
    df_escaladas = pd.DataFrame(scaler.fit_transform(df_predictoras),columns=columnas_a_escalar,index=df_final.index)

    df_final_estandarizado = pd.concat([df_final[no_escalar], df_escaladas],axis=1)

    return df_final_estandarizado, scaler

In [9]:
clientes_df = generar_variables_ingenieria(clientes_df)
clientes_df,requerimientos_df = imputacion_variables(clientes_df,requerimientos_df)
clientes_df, artifact_encoders_clientes = encoder_categoricos(clientes_df)
requerimientos_df = construir_variables_requerimientos(requerimientos_df)
df_final = clientes_df.merge(requerimientos_df, on='ID_CORRELATIVO', how='inner')
df_final, artifact_scaler = estandarizacion(df_final)

In [10]:
df_final

,ID_CORRELATIVO,CODMES,ATTRITION,ANTIGUEDAD,EDAD,FLAG_LIMA_PROVINCIA,FLG_BANCARIZADO,FLG_NOMINA,FLG_SDO_OTSSFF_MENOS0,FLG_SDO_OTSSFF_MENOS1,...,dictamen_NO PROCEDE,dictamen_PROCEDE PARCIAL,dictamen_PROCEDE TOTAL,nro_dictamenes,nro_productos_servicios,nro_submotivos,nro_tipos_requerimiento,tipo_Reclamo,tipo_Solicitud,total_requerimientos
0,35653,201208,0,0.366230,-0.905107,0.558367,0.269730,-0.862748,0.799069,-1.239641,...,0.361984,-0.269447,-0.969657,-0.404097,-0.377633,-0.450505,-0.369419,-0.072400,-0.651024,-0.481904
1,56800,201208,0,-0.342512,-0.010832,-1.790938,0.269730,1.159088,-1.251456,-1.239641,...,-0.762521,3.438779,-0.969657,-0.404097,-0.377633,-0.450505,-0.369419,-0.072400,-0.651024,-0.481904
2,8410,201208,0,0.011859,2.870723,-1.790938,0.269730,-0.862748,0.799069,0.806685,...,0.361984,-0.269447,-0.969657,-0.404097,-0.377633,-0.450505,-0.369419,-1.125015,0.968408,-0.481904
3,26877,201208,0,0.720601,-0.805743,0.558367,0.269730,1.159088,-1.251456,0.806685,...,0.361984,-0.269447,0.377027,2.154089,2.119972,1.387971,-0.369419,0.980215,-0.651024,0.541704
4,89278,201208,0,0.011859,0.386624,0.558367,0.269730,1.159088,-1.251456,-1.239641,...,-0.762521,-0.269447,0.377027,-0.404097,-0.377633,-0.450505,-0.369419,-0.072400,-0.651024,-0.481904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28084,98207,201208,0,0.720601,-1.004471,0.558367,-3.707409,-0.862748,-1.251456,-1.239641,...,0.361984,-0.269447,0.377027,2.154089,2.119972,1.387971,2.706954,-0.072400,0.968408,0.541704
28085,95667,201208,0,-1.051253,-0.408288,0.558367,0.269730,-0.862748,0.799069,0.806685,...,-0.762521,-0.269447,0.377027,-0.404097,-0.377633,-0.450505,-0.369419,-1.125015,0.968408,-0.481904
28086,3896,201208,0,-0.342512,3.367542,-1.790938,0.269730,1.159088,0.799069,0.806685,...,0.361984,-0.269447,-0.969657,-0.404097,-0.377633,-0.450505,-0.369419,-1.125015,0.968408,-0.481904
28087,72060,201208,0,0.720601,-0.308924,0.558367,0.269730,-0.862748,0.799069,0.806685,...,0.361984,-0.269447,-0.969657,-0.404097,-0.377633,-0.450505,-0.369419,-0.072400,-0.651024,-0.481904
